In [3]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

In [4]:
#change bike_lanes file path to citibike-publicspace
bike_lanes = gpd.read_file('/Users/kristikorsberg/Documents/nyc_bike_routes_2016.shp')
stations = pd.read_csv('../data/external/citibike_station_info.csv')

In [5]:
bike_lanes.columns

Index([  u'OBJECTID',  u'SegmentID', u'allclasses',    u'bikedir',
             u'boro',   u'comments', u'fromstreet', u'ft_facilit',
         u'geometry',   u'instdate',  u'lanecount',    u'moddate',
          u'onoffst',     u'street', u'tf_facilit',   u'tostreet'],
      dtype='object')

In [6]:
bike_lanes = bike_lanes.drop(['instdate', 'moddate'], 1)

In [7]:
bike_lanes = bike_lanes.to_crs({'init': 'epsg:4326'})

In [8]:
geometry = gpd.GeoSeries([Point(xy) for xy in zip(stations.Longitude, stations.Latitude)])
geometry = geometry.buffer(.0005)
geo_stations = gpd.GeoDataFrame(stations, geometry=geometry)
geo_stations.crs = {'init' :'epsg:4326'}
geo_stations.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry
0,72,W 52 St & 11 Ave,W 52 St & 11 Ave,40.767272,-73.993929,"POLYGON ((-73.99342888 40.76727216, -73.993431..."
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74...."
2,82,St James Pl & Pearl St,St James Pl & Pearl St,40.711174,-74.000165,"POLYGON ((-73.99966544999999 40.71117416, -73...."
3,83,Atlantic Ave & Fort Greene Pl,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,"POLYGON ((-73.97582328 40.68382604, -73.975825..."
4,116,W 17 St & 8 Ave,W 17 St & 8 Ave,40.741776,-74.001497,"POLYGON ((-74.00099745999999 40.74177603, -74...."


In [9]:
stations_bike_quality = gpd.sjoin(geo_stations, bike_lanes, op='intersects')
stations_bike_quality.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,index_right,OBJECTID,SegmentID,allclasses,bikedir,boro,comments,fromstreet,ft_facilit,lanecount,onoffst,street,tf_facilit,tostreet
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9448,9490,0031767,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9451,9493,0031796,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9488,9530,0110911,II,L,1.0,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9450,9492,0031798,II,L,1.0,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9487,9529,0031789,II,L,1.0,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY


In [10]:
#bike direction
ft_facilit_keys = stations_bike_quality['ft_facilit'].unique()
tf_facilit_keys = stations_bike_quality['tf_facilit'].unique()
print(ft_facilit_keys)
print(tf_facilit_keys)

[None u'Protected Path' u'Bike-Friendly Parking' u'Curbside' u'Standard'
 u'Sharrows' u'Greenway' u'Sidewalk' u'Ped Plaza' u'Signed Route'
 u'Opposite Sidewalk']
[u'Standard' None u'Bike-Friendly Parking' u'Curbside' u'Sharrows'
 u'Protected Path' u'Ped Plaza' u'Greenway' u'Sidewalk'
 u'Sharrows/Protected Path' u'Signed Route' u'<Null>' u'Sharrows/Standard']


ft_facility = feature facility start point to end point

bikedirR = ride start point to end point

tf_facility = feature facility end point to start point

bikedirL = ride end point to start point

bikedir2 = two-way

protected bike lanes: http://www.nyc.gov/html/dot/downloads/pdf/2014-09-03-bicycle-path-data-analysis.pdf

standard, signed route, sharrows: http://bikingrules.org/biking/laneprimer/

In [11]:
ft_score = []
for row in stations_bike_quality['ft_facilit']:
    if row == 'Greenway':
        ft_score.append(5)
    elif row == 'Signed Route':
        ft_score.append(5)
    elif row == 'Ped Plaza':
        ft_score.append(5)
    elif row == 'Protected Path':
        ft_score.append(4)
    elif row == 'Bike-Friendly Parking':
        ft_score.append(4)
    elif row == 'Curbside':
        ft_score.append(3)
    elif row == 'Standard':
        ft_score.append(3)
    elif row == 'Sidewalk':
        ft_score.append(2)
    elif row == 'Opposite Sidewalk':
        ft_score.append(2)
    elif row == 'Sharrows':
        ft_score.append(1)
    else:
        ft_score.append(0)
stations_bike_quality['ft_facilit_score'] = ft_score

In [12]:
tf_score = []
for row in stations_bike_quality['tf_facilit']:
    if row == 'Greenway':
        tf_score.append(5)
    elif row == 'Signed Route':
        tf_score.append(5)
    elif row == 'Ped Plaza':
        tf_score.append(5)
    elif row == 'Protected Path':
        tf_score.append(4)
    elif row == 'Bike-Friendly Parking':
        tf_score.append(4)
    elif row == 'Curbside':
        tf_score.append(3)
    elif row == 'Standard':
        tf_score.append(3)
    elif row == 'Sharrows/Protected Path':
        tf_score.append(2.5)
    elif row == 'Sidewalk':
        tf_score.append(2)
    elif row == 'Opposite Sidewalk':
        tf_score.append(2)
    elif row == 'Sharrows/Standard':
        tf_score.append(1.5)
    elif row == 'Sharrows':
        tf_score.append(1)
    else:
        tf_score.append(0)
stations_bike_quality['tf_facilit_score'] = tf_score

In [13]:
stations_bike_quality.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,index_right,OBJECTID,SegmentID,allclasses,...,comments,fromstreet,ft_facilit,lanecount,onoffst,street,tf_facilit,tostreet,ft_facilit_score,tf_facilit_score
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9448,9490,0031767,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9451,9493,0031796,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9488,9530,0110911,II,...,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9450,9492,0031798,II,...,None,BEACH ST,None,1.0,ON,W BROADWAY,Standard,READE ST,0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",9487,9529,0031789,II,...,None,BEACH ST,None,1.0,ON,VARICK ST,Standard,W BROADWAY,0,3.0


In [14]:
stations_bike_cut = stations_bike_quality.drop(['index_right', 'bikedir', 'OBJECTID', 'allclasses', 'comments', 'fromstreet', 'onoffst', 'street', 'tostreet'], axis=1)
#stations_bike_cut.head()
print(len(stations_bike_cut))

1049


In [15]:
nums = []
nums2 = []
for elem in stations_bike_cut['ft_facilit_score']:
    nums.append(elem)
#print(nums)

for elem in stations_bike_cut['tf_facilit_score']:
    nums2.append(elem)
#print(nums2)

nums_added = []
for i in range(0,1049):
    if nums[i] > 0 and nums2[i] > 0:
        nums_added.append((nums[i] + nums2[i]) / 2)
    elif nums[i] > 0 and nums2[i] == 0:
        nums_added.append(nums[i])
    elif nums[i] == 0 and nums2[i] > 0:
        nums_added.append(nums2[i])

#print(nums_added)
stations_bike_cut['metric'] = nums_added
stations_bike_cut.head()

,Station_id,Station_Name,Location,Latitude,Longitude,geometry,SegmentID,boro,ft_facilit,lanecount,tf_facilit,ft_facilit_score,tf_facilit_score,metric
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031767,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031796,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0110911,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031798,1.0,None,1.0,Standard,0,3.0,3.0
1,79,Franklin St & W Broadway,Franklin St & W Broadway,40.719116,-74.006667,"POLYGON ((-74.00616660999999 40.71911552, -74....",0031789,1.0,None,1.0,Standard,0,3.0,3.0


In [ ]:
#groupby and take the average of the station

In [37]:
stations_bikes = pd.DataFrame(stations_bike_cut['metric'].groupby([stations_bike_cut['Station_id']]).mean())
stations_bikes.reset_index()

,Station_id,metric
0,79,3.000000
1,116,4.000000
2,119,4.000000
3,120,4.000000
4,127,4.000000
5,128,3.000000
6,143,3.000000
7,144,3.000000
8,146,3.000000
9,147,3.000000
